In [19]:
import requests, os
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import re
from calendar import monthrange
from collections import Counter
from dateutil.relativedelta import relativedelta
from collections import Counter, defaultdict
import numpy as np

## HTTPヘッダー設定

In [20]:
HEADERS = {
    'User-Agent': (
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
        'AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/113.0.0.0 Safari/537.36'
    ),
    'Referer': 'https://min-repo.com/',
    'Accept-Language': 'ja-JP,ja;q=0.9'
}

In [21]:

# --- セッション初期化 & キャッシュ設定 (高速化) ---
try:
    import requests_cache
    requests_cache.install_cache('minrepo_cache', expire_after=60*60*24, backend='sqlite')
    CACHE_ENABLED = True
except ImportError:
    CACHE_ENABLED = False

session = requests.Session()
session.headers.update(HEADERS)


## 動的イベント日リスト生成関数

In [22]:
def generate_event_dates_priority(base_date):
    end = base_date
    start = end - timedelta(days=364)
    day = base_date.day
    digit = day % 10
    weekday = base_date.weekday()
    # 月内の基準日の曜日出現順を算出
    month_wdays = [d for d in range(1, monthrange(base_date.year, base_date.month)[1]+1)
                   if datetime(base_date.year, base_date.month, d).weekday() == weekday]
    occurrence = month_wdays.index(day) + 1 if day in month_wdays else None

    prioritized = {}
    m = start.replace(day=1)
    while m <= end:
        y, mo = m.year, m.month
        dim = monthrange(y, mo)[1]
        # 当月の同曜日リスト
        wdays = [d for d in range(1, dim+1)
                 if datetime(y, mo, d).weekday() == weekday]

        for d in range(1, dim+1):
            dt = datetime(y, mo, d)
            if not (start <= dt <= end):
                continue

            # ここで優先度を変更
            if dt.day == day:
                rule = '日付'
            elif occurrence and len(wdays) >= occurrence and dt.day == wdays[occurrence-1]:
                # 例: '第2月曜日'
                rule = f'第{occurrence}曜日'
            elif dt.day % 10 == digit:
                rule = 'Xのつく日'
            elif dt.weekday() == weekday:
                rule = '曜日'
            else:
                rule = None

            if rule and dt not in prioritized:
                prioritized[dt] = rule

        # 次月へ
        m = (m.replace(day=28) + timedelta(days=4)).replace(day=1)
    return prioritized


## データ取得 & 集計関数

In [23]:
def parse_posts(html):
    soup = BeautifulSoup(html, 'html.parser')
    records = []
    for post in soup.select('div.ichiran_post'):
        # ── 投稿ページ URL ──
        title_tag = post.select_one('div.ichiran_title a')
        url       = title_tag['href'] if title_tag and title_tag.has_attr('href') else None

        # ── 以下、既存のavg_diffやdetails取得など ──
        result = post.select_one('div.ichiran_result')
        m = re.search(r'平均差枚：\+?([0-9,]+)', result.text) if result else None
        if not m:
            continue
        val = int(m.group(1).replace(',', ''))
        if val < 300:
            continue

        details = []
        result2 = post.select_one('div.ichiran_result2')
        if result2:
            details = [line.strip() for line in result2.get_text(separator="\\n").split("\\n") if line.strip()]

        records.append({
            'hall'     : post.select_one('div.hall_header span.hall_name a').text.strip(),
            'url'      : url,               # ← ここで追加
            'avg_diff' : val,
            'details'  : details,
        })
    return records


def get_hot_halls_once(date_str):
    url = f'https://min-repo.com/category/%e6%9d%b1%e4%ba%ac%e9%83%bd/?report_date={date_str}&orderby=heikin_samai'
    recs = []
    while url:
        r = session.get(url)
        r.raise_for_status()
        recs += parse_posts(r.text)
        soup = BeautifulSoup(r.text, 'html.parser')
        nxt = soup.select_one('.wp-pagenavi a.nextpostslink')
        url = nxt['href'] if nxt else None
    return pd.DataFrame(recs)


In [24]:
if __name__ == '__main__':
    
    TOP_HALLS = 10          # ← ここを変えれば自由に調整できる
    TOP_RANKS = 5          # 機種・末尾を何位まで表示するか
    
    # ハイパーパラメータ（Notebook内で調整可能）
    alpha = 0.5
    beta  = 0.3
    gamma = 0.2
    
    input_str = input('基準日 (YYYY-MM-DD) を入力してください: ')
    base_date = datetime.strptime(input_str, '%Y-%m-%d')
    # prioritized_map: {datetime→ルール文字列}
    prioritized_map = generate_event_dates_priority(base_date)
    # ─── 「第N曜日」判定用の occurrence を再計算 ───
    day = base_date.day
    weekday = base_date.weekday()
    # その月の該当曜日リスト
    month_wdays = [
        d for d in range(1, monthrange(base_date.year, base_date.month)[1]+1)
        if datetime(base_date.year, base_date.month, d).weekday() == weekday
    ]
    occurrence = month_wdays.index(day) + 1 if day in month_wdays else None
    # ────────────────────────────────────────────────
    # 優先度の高い順に並び替えた日付リストを取得
    event_dates = sorted(prioritized_map.keys())
    # スクレイピングと集計
    all_dfs = []
    for dt in event_dates:
        df = get_hot_halls_once(dt.strftime('%Y-%m-%d'))
        df['date'] = dt.date()
        all_dfs.append(df)
    all_df = pd.concat(all_dfs, ignore_index=True)

    # 2) 出現回数ランキング
    freq = all_df['hall'].value_counts().reset_index()
    freq.columns = ['hall','frequency']
    freq.insert(0, 'rank', range(1, len(freq)+1))
    print('\n② 出現回数ランキング(上位10):')
    display(freq.head(TOP_HALLS))


# ── ホール×日付ごとの出現回数を集計 ──
date_counts = (
    all_df
    .groupby(['hall','date'])
    .size()
    .reset_index(name='event_count')
)

# 日付 → ルール のマッピング
rule_map = {dt.date(): rule for dt, rule in prioritized_map.items()}

# date_counts に 'rule' カラムを追加
date_counts['rule'] = date_counts['date'].map(rule_map)

# 出現回数ランキング上位 n ホールの取得
top_halls = freq.head(TOP_HALLS)['hall'].tolist()

for hall in top_halls:
    print("\n----------------------------------------------------------------------------------------------------------------------------------------------------")
    print('\n出現回数ランキング上位3ホール 内訳（日付／第N曜日／Xのつく日／曜日）:')
    df_h = date_counts[date_counts['hall'] == hall]
    # ここで並び順を ['日付', '第N曜日', 'Xのつく日', '曜日'] に
    breakdown = df_h.groupby('rule')['event_count'].sum().reindex(
        # ここで occurrence を使って '第N曜日' を生成
        ['日付', f'第{occurrence}曜日', 'Xのつく日', '曜日'],
        fill_value=0
    )
    
    # 表示
    print(f"\n{hall} (合計 {freq[freq['hall']==hall]['frequency'].iloc[0]} 回):")
    for rule, cnt in breakdown.items():
        print(f"  {rule:<12}: {cnt} 回")
    
    subset = all_df[all_df['hall'] == hall]

    # ─── 3) 末尾番号（1年分） & 機種名（直近6か月） ───
    tail_list, machine_list = [], []
    for details in subset['details']:
        for item in details:
            txt = item.lstrip('☆◎▲◯').strip()
            if txt.startswith('末尾'):
                tail_list.append(txt[len('末尾'):])
            else:
                machine_list.append(txt)

    # ─── 4) 頻度テーブル ───
    tail_df = (
        pd.DataFrame(Counter(tail_list).most_common(TOP_RANKS),
                     columns=['末尾', 'count'])
        .assign(rank=lambda df: range(1, len(df)+1),
                frequency_pct=lambda df: (df['count']/df['count'].sum()*100).round(1).astype(str)+'%')
        .loc[:, ['rank','末尾','frequency_pct']]
    )

    machine_df = (
        pd.DataFrame(Counter(machine_list).most_common(TOP_RANKS),
                     columns=['機種名','count'])
        .assign(rank=lambda df: range(1, len(df)+1),
                frequency_pct=lambda df: (df['count']/df['count'].sum()*100).round(1).astype(str)+'%')
        .loc[:, ['rank','機種名','frequency_pct']]
    )

    # ─── 5) 精度重視：末尾番号スコア分析 (recency × avg diff) ───
    tail_scores = defaultdict(float)
    for _, row in subset.iterrows():
        days_diff         = (base_date.date() - row['date']).days
        recency_weight    = 1.0 / (days_diff + 1)
        performance_weight = row['avg_diff'] / 300.0
        weight            = recency_weight * performance_weight

        for item in row['details']:
            txt = item.lstrip('☆◎▲◯').strip()
            if txt.startswith('末尾'):
                tail_num = txt[len('末尾'):]
                tail_scores[tail_num] += weight

    tail_pred_df = (
        pd.DataFrame(
            sorted(tail_scores.items(), key=lambda x: x[1], reverse=True)[:TOP_RANKS],
            columns=['末尾','score']
        )
        .assign(rank=lambda df: range(1, len(df)+1),
                weight_pct=lambda df: (df['score']/df['score'].sum()*100).round(1).astype(str)+'%')
        .loc[:, ['rank','末尾','score','weight_pct']]
    )

    # ─── 6) 表示 ───
        
    print(f"\n=== {hall} の末尾番号頻度 (年間) ===")
    display(tail_df)

    print(f"\n=== {hall} の機種傾向 (直近6か月) ===")
    display(machine_df)

    print(f"\n=== {hall} の末尾番号傾向分析／高設定期待末尾 (直近6か月) ===")
    display(tail_pred_df)
    
    # ── 1) ホール詳細ページから設置台数・台番一覧を取得 (event_dates 全日分) ──
    install_counts = {}  # 機種名→累計設置台数
    positions      = {}  # 機種名→累計台番リスト

    # subset は all_df[all_df['hall']==hall]
    for page_url in subset['url'].dropna().unique():
        try:
            res = session.get(page_url)
            res.raise_for_status()
            res.encoding = 'UTF-8'
            soup = BeautifulSoup(res.text, 'html.parser')
        except Exception as e:
            print(f"URL取得エラー {page_url}: {e}")
            continue

        # (A) 「2台以上設置機種」テーブルから勝率の分母を累計
        header = soup.find('h2', text=re.compile(r'2台以上設置機種'))
        if header:
            tbl = header.find_next('table')
            for tr in tbl.find_all('tr')[1:]:
                tds = tr.find_all('td')
                if len(tds) < 4:
                    continue
                name    = tds[0].get_text(strip=True)
                winrate = tds[3].get_text(strip=True)  # "○/△"
                cnt     = int(winrate.split('/')[-1]) if '/' in winrate else 0
                install_counts[name] = install_counts.get(name, 0) + cnt

        # (B) 全台データ一覧 (?kishu=all) から台番を累積
        all_url = page_url.rstrip('/') + '?kishu=all'
        try:
            res2  = session.get(all_url)
            res2.raise_for_status()
            res2.encoding = 'UTF-8'
            soup2 = BeautifulSoup(res2.text, 'html.parser')
        except:
            continue

        for tr in soup2.select('table tr'):
            tds = tr.find_all('td')
            if len(tds) < 2:
                continue
            name     = tds[0].get_text(strip=True)
            num_text = tds[1].get_text(strip=True)
            if num_text.isdigit():
                positions.setdefault(name, []).append(int(num_text))

    # ホール全体の総設置台数
    total_install = sum(install_counts.values()) or 1

    # 2) 基本スコア（recency×performance）を集計
    machine_base_scores = defaultdict(float)
    for _, row in subset.iterrows():
        days_diff = (base_date.date() - row['date']).days
        recency_weight    = 1.0 / (days_diff + 1)
        performance_weight = row['avg_diff'] / 300.0
        base_weight       = recency_weight * performance_weight
        for item in row['details']:
            name = item.lstrip('☆◎▲◯').strip()
            if not name or name.startswith('末尾'):
                continue
            machine_base_scores[name] += base_weight

    # 3) install_ratio, spread_factor, volatility_factor を計算し、統合スコアに反映
    machine_scores = {}
    for name, base_score in machine_base_scores.items():
        # install_ratio（設置比率）
        install_count  = install_counts.get(name, 1)
        install_ratio  = install_count / total_install

        # spread_factor（台番の分散／配置幅）
        pos_list = positions.get(name, [])
        if pos_list:
            var   = float(np.var(pos_list))
            rng   = max(pos_list) - min(pos_list) + 1
            spread = var / rng
        else:
            spread = 0.0

        # volatility_factor（過去平均差枚の変動係数）
        diffs = [
            r['avg_diff']
            for _, r in subset.iterrows()
            if any(item.lstrip('☆◎▲◯').strip() == name for item in r['details'])
        ]
        if len(diffs) > 1 and np.mean(diffs) != 0:
            volatility = np.std(diffs, ddof=0) / np.mean(diffs)
        else:
            volatility = 0.0

        # 統合スコア算出
        score = (
            base_score
            * (1 + alpha * install_ratio)
            * (1 + beta  * spread)
            * (1 - gamma * volatility)
        )
        machine_scores[name] = score

    # 4) 結果を DataFrame にまとめる（既存の weight_pct計算と同様）
    total_score = sum(machine_scores.values()) or 1.0
    machine_pred_df = (
        pd.DataFrame(sorted(machine_scores.items(), key=lambda x: x[1], reverse=True)[:TOP_RANKS],
                    columns=['機種名','score'])
        .assign(rank=lambda df: range(1, len(df)+1),
                weight_pct=lambda df: (df['score']/total_score*100).round(1).astype(str)+'%')
        .loc[:, ['rank','機種名','score','weight_pct']]
    )
    
    print(f"\n=== {hall} の高設定期待機種予測（スコアリングモデル）（{base_date.date()}） ===")
    display(machine_pred_df)


② 出現回数ランキング(上位10):


,rank,hall,frequency
0,1,エスパス日拓渋谷本館,32
1,2,アミューズ浅草店,23
2,3,エスパス日拓西武新宿駅前,21
3,4,エクスアリーナ東京,20
4,5,アイランド秋葉原店,18
5,6,グランパ大久保,15
6,7,エスパス日拓秋葉原駅前店,12
7,8,マルハン池袋SLOT BASE,12
8,9,グランパ中野,12
9,10,大山オーシャン,11



----------------------------------------------------------------------------------------------------------------------------------------------------

出現回数ランキング上位3ホール 内訳（日付／第N曜日／Xのつく日／曜日）:

エスパス日拓渋谷本館 (合計 32 回):
  日付          : 0 回
  第3曜日        : 6 回
  Xのつく日       : 3 回
  曜日          : 23 回

=== エスパス日拓渋谷本館 の末尾番号頻度 (年間) ===


,rank,末尾,frequency_pct
0,1,ゾロ目,21.4%
1,2,6,21.4%
2,3,5,21.4%
3,4,1,21.4%
4,5,8,14.3%



=== エスパス日拓渋谷本館 の機種傾向 (直近6か月) ===


...


=== 大山オーシャン の末尾番号傾向分析／高設定期待末尾 (直近6か月) ===


,rank,末尾,score,weight_pct
0,1,8,0.136689,44.5%
1,2,2,0.135840,44.3%
2,3,1,0.016606,5.4%
3,4,6,0.011952,3.9%
4,5,9,0.005853,1.9%


C:\Temp\ipykernel_36544\917123648.py:156: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  header = soup.find('h2', text=re.compile(r'2台以上設置機種'))



=== 大山オーシャン の高設定期待機種予測（スコアリングモデル）（2025-05-18） ===


,rank,機種名,score,weight_pct
0,1,スマスロ モンスターハンターライズ,0.566783,15.3%
1,2,ファンキージャグラー２ＫＴ,0.434117,11.8%
2,3,Lパチスロからくりサーカス,0.367987,10.0%
3,4,新ハナビ,0.289687,7.8%
4,5,A-SLOT+ ディスクアップ ULTRAREMIX,0.262237,7.1%
